![](https://i.imgur.com/tzWwJLY.png)

# Competition Baseline - Object Detection + Classification

**Welcome!** This notebook provides a simple YOLOv8 baseline to get you started.

## What this notebook does:
1. ✅ Loads and prepares the dataset
2. ✅ Trains a YOLOv8 object detection model
3. ✅ Generates a valid submission file
4. ✅ Uses "-" for images with no detections (required format)

## Competition Metric:
**Final Score = 0.5 × mAP@[0.5:0.95] + 0.5 × F1-Score**

---

In [ ]:
# Install required packages
!pip install ultralytics -q

In [ ]:
# Import libraries
import os
import shutil
import pandas as pd
from pathlib import Path
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
from tqdm import tqdm

## 📁 Configuration
Update `ROOT_DIR` to point to your dataset location

In [ ]:
# Update this path to match your dataset
ROOT_DIR = "/kaggle/input/dal-shemagh-detection-challenge"
WORK_DIR = "./yolo_data"

print(f"Dataset directory: {ROOT_DIR}")
print(f"Working directory: {WORK_DIR}")

## 📊 Load Data

In [ ]:
# Load training labels
train_df = pd.read_csv(os.path.join(ROOT_DIR, "train_labels.csv"))
print(f"Training images: {len(train_df)}")
print("\nFirst few rows:")
display(train_df.head())

## ✂️ Train/Validation Split
Split data 80/20 with stratification by category

In [ ]:
# Split into train (80%) and validation (20%)
train_data, val_data = train_test_split(
    train_df,
    test_size=0.2,
    random_state=42,
)

print(f"Train set: {len(train_data)} images")
print(f"Validation set: {len(val_data)} images")

## 🗂️ Prepare YOLO Directory Structure
YOLO expects images and labels in specific folders

In [ ]:
# Create directories
for split in ['train', 'val']:
    os.makedirs(f"{WORK_DIR}/images/{split}", exist_ok=True)
    os.makedirs(f"{WORK_DIR}/labels/{split}", exist_ok=True)

def prepare_yolo_split(df, split_name):
    """Copy images and labels to YOLO format directories"""
    print(f"\nPreparing {split_name} set...")
    
    for _, row in tqdm(df.iterrows(), total=len(df)):
        filename = row['filename']
        stem = Path(filename).stem
        
        # Copy image
        src_img = os.path.join(ROOT_DIR, "images/train", filename)
        dst_img = os.path.join(WORK_DIR, f"images/{split_name}", filename)
        shutil.copy(src_img, dst_img)
        
        # Copy label (create empty if doesn't exist)
        src_lbl = os.path.join(ROOT_DIR, "labels/train", f"{stem}.txt")
        dst_lbl = os.path.join(WORK_DIR, f"labels/{split_name}", f"{stem}.txt")
        
        if os.path.exists(src_lbl):
            shutil.copy(src_lbl, dst_lbl)
        else:
            # Empty label = no objects in image
            open(dst_lbl, 'w').close()

# Prepare both splits
prepare_yolo_split(train_data, 'train')
prepare_yolo_split(val_data, 'val')

print("\n✓ YOLO directories ready!")

## ⚙️ Create YOLO Configuration

In [ ]:
# Create data.yaml config file
yaml_content = f"""path: {os.path.abspath(WORK_DIR)}
train: images/train
val: images/val

nc: 2
names: ['Face', 'Shemagh']
"""

with open(f"{WORK_DIR}/data.yaml", "w") as f:
    f.write(yaml_content)

print("✓ Configuration created!")
print("\nContent:")
print(yaml_content)

## 🚀 Train YOLO Model

Training YOLOv8n (nano) - fastest model, good baseline.

**To improve:** Try `yolov8s.pt`, `yolov8m.pt`, or `yolov8l.pt` for better accuracy

In [ ]:
import albumentations as A

imgtransform = A.Compose([
    # A.Resize((640, 640)),
    A.HorizontalFlip(p=0.5),
    A.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    A.GaussianBlur(p=0.1),
    A.Rotate(limit=10),
])

In [ ]:
import torch
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))


In [ ]:
# Initialize model
model = YOLO('yolo11s.pt')

results = model.train(
    data=f"{WORK_DIR}/data.yaml",
    epochs=200,
    imgsz=640,
    batch=16,
    patience=50,
    device = 'cuda' ,

    # LR tuning
    cos_lr=True,
    lr0=0.01,
    lrf=0.01,

    # ---- Augmentations ----
    hsv_h=0.015,     # color hue jitter
    hsv_s=0.7,       # saturation jitter
    hsv_v=0.4,       # brightness jitter

    degrees=5.0,     # small rotations
    translate=0.10,  # shift
    scale=0.50,      # zoom in/out
    shear=2.0,       # shear
    perspective=0.0005,
    bgr=0.35, 

    fliplr=0.5,      # horizontal flip

    mosaic=1.0,      # strong detection booster
    mixup=0.90, 
    cutmix = 0.90,
    copy_paste=0.1,
    copy_paste_mode = 'mixup',

    close_mosaic=10, # turn off mosaic for last 10 epochs (often improves final quality)

    save=True,
    plots=True,
    verbose=True, 
    augmentations = imgtransform,
)

print("\n✓ Training complete!")


In [ ]:
def calculate_iou(box1, box2):
    """
    Calculate IoU between two boxes (x, y, w, h normalized).
    """
    # Convert from xywh to x1, y1, x2, y2
    b1_x1, b1_x2 = box1[0] - box1[2]/2, box1[0] + box1[2]/2
    b1_y1, b1_y2 = box1[1] - box1[3]/2, box1[1] + box1[3]/2
    
    b2_x1, b2_x2 = box2[0] - box2[2]/2, box2[0] + box2[2]/2
    b2_y1, b2_y2 = box2[1] - box2[3]/2, box2[1] + box2[3]/2

    # Intersection coordinates
    inter_x1 = max(b1_x1, b2_x1)
    inter_y1 = max(b1_y1, b2_y1)
    inter_x2 = min(b1_x2, b2_x2)
    inter_y2 = min(b1_y2, b2_y2)

    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    b1_area = (b1_x2 - b1_x1) * (b1_y2 - b1_y1)
    b2_area = (b2_x2 - b2_x1) * (b2_y2 - b2_y1)
    
    union_area = b1_area + b2_area - inter_area
    return inter_area / union_area if union_area > 0 else 0


## 📈 Validate Model

In [ ]:
# Evaluate on validation set
metrics = model.val()

print("\n" + "="*60)
print("VALIDATION RESULTS")
print("="*60)
print(f"mAP@0.5:0.95: {metrics.box.map:.4f}")
print(f"mAP@0.5:     {metrics.box.map50:.4f}")
print(f"mAP@0.75:    {metrics.box.map75:.4f}")
print("="*60)

## 🔮 Generate Submission

Run inference on test set and create submission file

In [ ]:
print("Generating submission file...\n")

test_img_dir = os.path.join(ROOT_DIR, "images/test")
submission_rows = []

# ===== DEBUG TOGGLES =====
DEBUG_PRINT = True
DEBUG_VIS   = True
DEBUG_N     = None
# =========================

test_images = sorted([f for f in os.listdir(test_img_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
print(f"Found {len(test_images)} test images\n")

for idx, filename in enumerate(tqdm(test_images, desc="Inference")):
    img_path = os.path.join(test_img_dir, filename)
    
    results = model.predict(
        source=img_path,
        device = 0,
        conf=0.15,
        iou=0.8,
        verbose=False
    )[0]

    pred_parts = []
    person_boxes = []
    shemagh_boxes = []

    if len(results.boxes) == 0:
        prediction_string = "-"
        right_place = 0
    else:
        for box in results.boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            x, y, w, h = box.xywhn[0].tolist()

            pred_parts.append(f"{cls} {conf:.4f} {x:.4f} {y:.4f} {w:.4f} {h:.4f}")

            if cls == 0:
                person_boxes.append([x, y, w, h])
            elif cls == 1:
                shemagh_boxes.append([x, y, w, h])

        prediction_string = " ".join(pred_parts)
        right_place = 0

        if person_boxes and shemagh_boxes:
            for s_box in shemagh_boxes:
                for p_box in person_boxes:
                    iou = calculate_iou(s_box, p_box)
                    if iou > 0.1:
                        right_place = 1
                        break
                if right_place == 1:
                    break


    if DEBUG_PRINT and (DEBUG_N is None or idx < DEBUG_N):
        print(f"\nImage: {filename}")
        print(f"Right Place: {right_place}")
    
        if prediction_string == "-":
            print("No detections")
        else:
            for part in pred_parts:
                print(part)
    
    # ===== DEBUG VISUALIZATION =====
    if DEBUG_VIS and (DEBUG_N is None or idx < DEBUG_N):
        import cv2
        import matplotlib.pyplot as plt
    
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls = int(box.cls[0])
            conf = float(box.conf[0])
    
            label = f"{cls}:{conf:.2f}"
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(
                img, label, (x1, max(0, y1 - 8)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2
            )
    plt.figure(figsize=(7, 7))
    plt.title(f"{filename} | Right Place: {right_place}")
    plt.axis("off")
    plt.imshow(img)
    plt.show()
    # ==========================================

    submission_rows.append({
        "filename": filename,
        "right_place": right_place,
        "prediction_string": prediction_string
    })

submission = pd.DataFrame(submission_rows)

print("\n✓ Submission generated!")
print(f"Total rows: {len(submission)}")
print(f"With detections: {(submission['prediction_string'] != '-').sum()}")
print(f"Without detections: {(submission['prediction_string'] == '-').sum()}")


In [ ]:
for idx, filename in enumerate(tqdm(test_images, desc="Inference")):
    img_path = os.path.join(test_img_dir, filename)

    if DEBUG_PRINT and (DEBUG_N is None or idx < DEBUG_N):
        print(f"\nImage: {filename}")
        print(f"Right Place: {right_place}")
    
        if prediction_string == "-":
            print("No detections")
        else:
            for part in pred_parts:
                print(part)
    
    # ===== DEBUG VISUALIZATION =====
    if DEBUG_VIS and (DEBUG_N is None or idx < DEBUG_N):
        import cv2
        import matplotlib.pyplot as plt
    
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls = int(box.cls[0])
            conf = float(box.conf[0])
    
            label = f"{cls}:{conf:.2f}"
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(
                img, label, (x1, max(0, y1 - 8)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2
            )
    plt.figure(figsize=(7, 7))
    plt.title(f"{filename} | Right Place: {right_place}")
    plt.axis("off")
    plt.imshow(img)
    plt.show()

In [ ]:

import cv2
import matplotlib.pyplot as plt

lowest_preds.sort(key=lambda x: x[0])  # lowest score first

N = 50  # change to show more/less
print(f"\nShowing {N} lowest-confidence images...")

for score, img_path, results in lowest_preds[:N]:
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # draw boxes
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        cls = int(box.cls[0])

        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(
            img,
            f"{cls}:{conf:.2f}",
            (x1, max(0, y1 - 8)),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (255, 0, 0),
            2
        )

    plt.figure(figsize=(7, 7))
    plt.title(f"{os.path.basename(img_path)} | lowest_conf={score:.3f}")
    plt.axis("off")
    plt.imshow(img)
    plt.show()


## 💾 Save Submission

In [ ]:
# Save to CSV
submission.to_csv("submission.csv", index=False)

print("\n" + "="*60)
print("✓ SUBMISSION SAVED: submission.csv")
print("="*60)
print("\nFirst 10 rows:")
display(submission.head(10))

---

## 🎯 Next Steps

This baseline gives you a working submission. To improve your score:

1. **Add Augmentations** - Try to add augmentations
2. **Use larger model** - Try `yolov8s.pt` or `yolov8m.pt`
3. **Train longer** - Increase epochs to 100-200
4. **Add classification model** - Build a model for `right_place` instead of always using 0 (or maybe think about a more elegant way to predict it!)

---

Good luck! 🍀